# Spectrograms - CNN Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import sqlalchemy
from sqlalchemy import create_engine, inspect

import math
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

## Import datasets

In [3]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'demographic',
 'diagnosis',
 'gval',
 'habits',
 'rval',
 'spectrogram']

In [4]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

## Preprocessing

### Define the target variable

In [5]:
# Isolate the diagnosis column
y = dataframes['diagnosis_df']['diagnosis'].copy()

# Encode the target variable, ignore subtype
y = y.apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Reshape the feature variables

In [6]:
# Input shape
width_px = 225
height_px = 166
num_channels = 4 # since RGBA

# Define inputs
input_shape = (height_px, width_px, num_channels)
input_reshape = (height_px, width_px)

In [22]:
# Dataframe order
rgba_order = ['rval_df', 'gval_df', 'bval_df', 'aval_df']

# Initialise list to hold the dataframes
rgba_df_list = []

# Loop through all the dataframes
for df in rgba_order:
    
    # Define the df columns
    df_cols = dataframes[df].columns

    # Reshape to its original dimensions
    data = np.array(
        [dataframes[df][col].values.reshape(input_reshape) for col in df_cols]
    )

    # Append to the list
    rgba_df_list.append(data)

# Define the feature variables
X = np.stack(rgba_df_list, axis=-1)

# Display the first for confirmation
X[0]

array([[[ 47,  17,  99, 255],
        [ 47,  17,  99, 255],
        [ 43,  16,  93, 255],
        ...,
        [ 35,  11,  70, 255],
        [ 47,  16,  90, 255],
        [ 49,  17,  93, 255]],

       [[ 45,  17,  97, 255],
        [ 45,  17,  98, 255],
        [ 41,  16,  90, 255],
        ...,
        [ 49,  15,  93, 255],
        [ 59,  16, 108, 255],
        [ 60,  15, 111, 255]],

       [[ 46,  17,  98, 255],
        [ 46,  17,  99, 255],
        [ 43,  16,  92, 255],
        ...,
        [ 29,  13,  68, 255],
        [ 51,  16,  99, 255],
        [ 54,  16, 104, 255]],

       ...,

       [[215,  69, 107, 255],
        [215,  69, 107, 255],
        [211,  67, 109, 255],
        ...,
        [ 76,  18, 120, 255],
        [184,  56, 115, 255],
        [201,  62, 114, 255]],

       [[235,  90,  96, 255],
        [235,  90,  96, 255],
        [231,  87,  98, 255],
        ...,
        [ 91,  20, 125, 255],
        [206,  70, 105, 255],
        [225,  77, 101, 255]],

       [[250

### Split and Scale

In [19]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [24]:
# Reshape the data
X_train_reshaped = X_train.reshape((
    X_train.shape[0], # total number of samples
    height_px * width_px * num_channels # total number flattened
))

X_test_reshaped = X_test.reshape((
    X_test.shape[0],
    height_px * width_px * num_channels
    ))

In [25]:
# Normalize training data to be between 0 and 1
X_scaler = MinMaxScaler()

# Scale the data
X_train_scaled = X_scaler.fit_transform(X_train_reshaped)
X_test_scaled = X_scaler.fit_transform(X_test_reshaped)

# Reshape the data back to the original
X_train_scaled = X_train_scaled.reshape((
    X_train_scaled.shape[0],
    height_px,
    width_px,
    num_channels
))

X_test_scaled = X_test_scaled.reshape((
    X_test_scaled.shape[0],
    height_px,
    width_px,
    num_channels
))

In [30]:
# Define the CNN model
cnn = Sequential()

# Add first convolutional layer
cnn.add(Conv2D(
    filters = 32,
    kernel_size = (3, 3),
    activation = 'relu',
    input_shape = (height_px, width_px, num_channels)
))

# Add first pooling layer
cnn.add(MaxPooling2D((2, 2)))

# Add second convolutional layer
cnn.add(Conv2D(
    filters = 64,
    kernel_size = (3, 3),
    activation = 'relu'
))

# Add second pooling layer
cnn.add(MaxPooling2D((2, 2)))

# Add third convolutional layer
cnn.add(Conv2D(
    filters = 128,
    kernel_size = (3, 3),
    activation = 'relu'
))

# Add third pooling layer
cnn.add(MaxPooling2D((2, 2)))

# Flatten the output before feeding into the fully connected layers
cnn.add(Flatten())

# Add dense layers for classification
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))  # Binary classification

# Display the summary
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 164, 223, 32)      1184      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 82, 111, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 80, 109, 64)       18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 40, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 38, 52, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 19, 26, 128)      

In [31]:
# Compile the model
cnn.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [32]:
# Train the model
cnn.fit(
    X_train_scaled,
    y_train,
    epochs = 15,
    shuffle = True, # reduce risk of overfitting
    verbose = 1
)

Epoch 1/15
5/5 [==============================] - 1s 236ms/step - loss: 1.2194 - accuracy: 0.5490
Epoch 2/15
5/5 [==============================] - 1s 224ms/step - loss: 0.6024 - accuracy: 0.7190
Epoch 3/15
5/5 [==============================] - 1s 224ms/step - loss: 0.5809 - accuracy: 0.7190
Epoch 4/15
5/5 [==============================] - 1s 222ms/step - loss: 0.6609 - accuracy: 0.7190
Epoch 5/15
5/5 [==============================] - 1s 224ms/step - loss: 0.6266 - accuracy: 0.7190
Epoch 6/15
5/5 [==============================] - 1s 234ms/step - loss: 0.5693 - accuracy: 0.7190
Epoch 7/15
5/5 [==============================] - 1s 228ms/step - loss: 0.5589 - accuracy: 0.7190
Epoch 8/15
5/5 [==============================] - 1s 224ms/step - loss: 0.5334 - accuracy: 0.7255
Epoch 9/15
5/5 [==============================] - 1s 223ms/step - loss: 0.5068 - accuracy: 0.7255
Epoch 10/15
5/5 [==============================] - 1s 225ms/step - loss: 0.4706 - accuracy: 0.7908
Epoch 11/15
5/5 [==

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = cnn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.7912 - accuracy: 0.7255 - 141ms/epoch - 70ms/step
Loss: 0.791204571723938, Accuracy: 0.7254902124404907


## Hyperparameter Tuning

__GUIDELINES__

- The number of filters for convolutional layers should:
    - Be a value to the power of 2
    - Increase for each following layer (i.e. `32`, `64`, `128`)
- The number of neurons for the fully connected layers should:
    - Be a value to the power of 2
    - Decrease for each following layer (i.e. `64`, `32`)

In [ ]:
# Define the model parameters


In [ ]:
def create_cnn_model(hp):
    cnn_model = Sequential()

    # Choose the number of convolutional layers
    